### Conversion of the CORINE land cover data to a useful format for air dispersion modelling
Author: Dr Scott Hamilton, Ricardo, 2016, scott.hamilton@ricardo.com

#### This python program reads the CORINE raster data which characterises land use across Europe at 100m resolution.

Each raster cell data is categorised using the following definitions which are then mapped to definitions
published by the USEPA for dispersion modelling- this allows albedo, roughness and Bowen ratio values to be mapped to the categories

1 - Continuous urban fabric

2 - Discontinuous urban fabric

3 - Industrial or commercial units

4 - Road and rail networks and associated land

5 - Port areas

6 - Airports

7 - Mineral extraction sites

8 - Dump sites

9 - Construction sites

10 - Green urban areas

11 - Sport and leisure facilities

12 - Non-irrigated arable land

13 - Permanently irrigated land

14 - Rice fields

15 - Vineyards

16 - Fruit trees and berry plantations

17 - Olive groves

18 - Pastures

19 - Annual crops associated with permanent crops

20 - Complex cultivation patterns

21 - Land principally occupied by agriculture with significant areas of natural vegetation

22 - Agro-forestry areas

23 - Broad-leaved forest

24 - Coniferous forest

25 - Mixed forest

26 - Natural grasslands

27 - Moors and heathland

28 - Sclerophyllous vegetation

29 - Transitional woodland-shrub

30 - Beaches - dunes - sands

31 - Bare rocks

32 - Sparsely vegetated areas

33 - Burnt areas

34 - Glaciers and perpetual snow

35 - Inland marshes

36 - Peat bogs

37 - Salt marshes

38 - Salines

39- Intertidal flats

40- Water courses

41- Water bodies

42- Coastal lagoons

43- Estuaries

44- Sea and ocean

48- NODATA

49- UNCLASSIFIED LAND SURFACE

50- UNCLASSIFIED WATER BODIES

255 - UNCLASSIFIED

The USEPA categories are as published in the AERMET and AERSURFACE user guides 
https://www3.epa.gov/scram001/7thconf/aermod/aermetugb.pdf
https://www3.epa.gov/scram001/7thconf/aermod/aersurface_userguide.pdf

The script below uses a series of logical tests to check the values stored in the raster cells before reassigning
them according to the test. E.g. if the raster cell has the value 1, this means "continuous urban" in the CORINE data.
This maps to the "High Intensity Residential" category in the USGS system which is adopted by the USEPA. Therefore if
a value of 1 is found, the appropriate values of roughnes, albedo and Bowen ratio are substituted into the data before
the other values (up to 44) are checked and substitutions made. 

When the subs have been made 3 new raster files are
written out with the data. These are then used in QGIS to estimate averages at the met stations using 1km and 10km
buffers as recommended by the USEPA.

The derived values can be used to set dispersion modelling parameters in a more reproducible manner. They also allow
more focussed sensitivity testing and can be used to describe conditions at the dispersion site also. The main issue
in that case would be the possibility of underestimating surface roughness which can be up to 3 in large, dense cities.

**The roughness length can therefore be treated as a low value which should not be reduced further.**

In [5]:
#------------------
# imports
#-----------------
import gdal
import numpy as np
import pandas as pd

In [6]:
#-----------------
# read CORINE raster
#-----------------
ds = gdal.Open("/Users/scottlynn73/Documents/CORINE/g100_clc12_V18_5_UK_ONLY.tif")
my_albedo_array = np.array(ds.GetRasterBand(1).ReadAsArray())
my_rough_array  = np.array(ds.GetRasterBand(1).ReadAsArray())
my_bowen_array  = np.array(ds.GetRasterBand(1).ReadAsArray())

In [7]:
#----------------
# replace the values in the array with the correct albedo values according to the classification in CORINE
#-----------------
my_albedo_array = np.where(my_albedo_array == 1, 0.22, my_albedo_array)  # substitute value for continuous urban
my_albedo_array = np.where(my_albedo_array == 2, 0.24, my_albedo_array)  # discontinuous urban
my_albedo_array = np.where(my_albedo_array == 3, 0.22, my_albedo_array)  # industrial / commercial
my_albedo_array = np.where(my_albedo_array == 4, 0.22, my_albedo_array)  # road/rail
my_albedo_array = np.where(my_albedo_array == 5, 0.22, my_albedo_array)  # ports
my_albedo_array = np.where(my_albedo_array == 6, 0.22, my_albedo_array)  # airports
my_albedo_array = np.where(my_albedo_array == 7, 0.30, my_albedo_array)  # mines
my_albedo_array = np.where(my_albedo_array == 8, 0.30, my_albedo_array)  # dumps
my_albedo_array = np.where(my_albedo_array == 9, 0.22, my_albedo_array)  # construction
my_albedo_array = np.where(my_albedo_array == 10, 0.26, my_albedo_array) # green urban
my_albedo_array = np.where(my_albedo_array == 11, 0.22, my_albedo_array) # sport/leisure
my_albedo_array = np.where(my_albedo_array == 12, 0.29, my_albedo_array) # non-irrigated arable
my_albedo_array = np.where(my_albedo_array == 13, 0.29, my_albedo_array) # irrigated arable
my_albedo_array = np.where(my_albedo_array == 14, 0.29, my_albedo_array) # rice fields
my_albedo_array = np.where(my_albedo_array == 15, 0.25, my_albedo_array) # vineyards
my_albedo_array = np.where(my_albedo_array == 16, 0.25, my_albedo_array) # fruit trees
my_albedo_array = np.where(my_albedo_array == 17, 0.25, my_albedo_array) # olive groves
my_albedo_array = np.where(my_albedo_array == 18, 0.29, my_albedo_array) # pastures
my_albedo_array = np.where(my_albedo_array == 19, 0.29, my_albedo_array) # crops
my_albedo_array = np.where(my_albedo_array == 20, 0.29, my_albedo_array) # complex cultivation
my_albedo_array = np.where(my_albedo_array == 21, 0.29, my_albedo_array) # agriculture
my_albedo_array = np.where(my_albedo_array == 22, 0.21, my_albedo_array) # agro-forest
my_albedo_array = np.where(my_albedo_array == 23, 0.25, my_albedo_array) # broad leaved forest
my_albedo_array = np.where(my_albedo_array == 24, 0.18, my_albedo_array) # evergreen forest
my_albedo_array = np.where(my_albedo_array == 25, 0.21, my_albedo_array) # mix forest
my_albedo_array = np.where(my_albedo_array == 26, 0.29, my_albedo_array) # grassland
my_albedo_array = np.where(my_albedo_array == 27, 0.29, my_albedo_array) # moors
my_albedo_array = np.where(my_albedo_array == 28, 0.29, my_albedo_array) # vegetation
my_albedo_array = np.where(my_albedo_array == 29, 0.25, my_albedo_array) # woodland shrub
my_albedo_array = np.where(my_albedo_array == 30, 0.30, my_albedo_array) # beaches/ sand
my_albedo_array = np.where(my_albedo_array == 31, 0.30, my_albedo_array) # bare rock
my_albedo_array = np.where(my_albedo_array == 32, 0.25, my_albedo_array) # sparse vegetation
my_albedo_array = np.where(my_albedo_array == 33, 0.25, my_albedo_array) # burnt areas
my_albedo_array = np.where(my_albedo_array == 34, 0.63, my_albedo_array) # glacier/ice
my_albedo_array = np.where(my_albedo_array == 35, 0.18, my_albedo_array) # inland marsh
my_albedo_array = np.where(my_albedo_array == 36, 0.18, my_albedo_array) # peat bog
my_albedo_array = np.where(my_albedo_array == 37, 0.18, my_albedo_array) # salt marsh
my_albedo_array = np.where(my_albedo_array == 38, 0.18, my_albedo_array) # salines
my_albedo_array = np.where(my_albedo_array == 39, 0.18, my_albedo_array) # intertidal flats
my_albedo_array = np.where(my_albedo_array == 40, 0.10, my_albedo_array) # water courses
my_albedo_array = np.where(my_albedo_array == 41, 0.10, my_albedo_array) # water bodies
my_albedo_array = np.where(my_albedo_array == 42, 0.10, my_albedo_array) # coastal lagoon
my_albedo_array = np.where(my_albedo_array == 43, 0.10, my_albedo_array) # estuaries
my_albedo_array = np.where(my_albedo_array == 44, 0.10, my_albedo_array) # sea and ocean

print(my_albedo_array.min())
print(my_albedo_array.max())

0.1
255.0


In [4]:
#----------------
# replace the roughness length values in the roughness array with the correct  values according to the classification in CORINE
#-----------------
my_rough_array = np.where(my_rough_array == 1, 1, my_rough_array) # substitute value for continuous urban
my_rough_array = np.where(my_rough_array == 2, 0.38, my_rough_array) # discontinuous urban
my_rough_array = np.where(my_rough_array == 3, 0.38, my_rough_array) # industrial / commercial
my_rough_array = np.where(my_rough_array == 4, 0.38, my_rough_array) # road/rail
my_rough_array = np.where(my_rough_array == 5, 0.70, my_rough_array) # ports
my_rough_array = np.where(my_rough_array == 6, 0.07, my_rough_array) # airports
my_rough_array = np.where(my_rough_array == 7, 0.05, my_rough_array) # mines
my_rough_array = np.where(my_rough_array == 8, 0.05, my_rough_array) # dumps
my_rough_array = np.where(my_rough_array == 9, 0.38, my_rough_array) # construction
my_rough_array = np.where(my_rough_array == 10, 0.02, my_rough_array) # green urban
my_rough_array = np.where(my_rough_array == 11, 0.38, my_rough_array) # sport/leisure
my_rough_array = np.where(my_rough_array == 12, 0.05, my_rough_array) # non-irrigated arable
my_rough_array = np.where(my_rough_array == 13, 0.05, my_rough_array) # irrigated arable
my_rough_array = np.where(my_rough_array == 14, 0.05, my_rough_array) # rice fields
my_rough_array = np.where(my_rough_array == 15, 0.23, my_rough_array) # vineyards
my_rough_array = np.where(my_rough_array == 16, 0.23, my_rough_array) # fruit trees
my_rough_array = np.where(my_rough_array == 17, 0.23, my_rough_array) # olive groves
my_rough_array = np.where(my_rough_array == 18, 0.09, my_rough_array) # pastures
my_rough_array = np.where(my_rough_array == 19, 0.09, my_rough_array) # crops
my_rough_array = np.where(my_rough_array == 20, 0.09, my_rough_array) # complex cultivation
my_rough_array = np.where(my_rough_array == 21, 0.09, my_rough_array) # agriculture
my_rough_array = np.where(my_rough_array == 22, 1.05, my_rough_array) # agro-forest
my_rough_array = np.where(my_rough_array == 23, 1.15, my_rough_array) # broad leaved forest
my_rough_array = np.where(my_rough_array == 24, 1.30, my_rough_array) # evergreen forest
my_rough_array = np.where(my_rough_array == 25, 1.15, my_rough_array) # mix forest
my_rough_array = np.where(my_rough_array == 26, 0.07, my_rough_array) # grassland
my_rough_array = np.where(my_rough_array == 27, 0.07, my_rough_array) # moors
my_rough_array = np.where(my_rough_array == 28, 0.07, my_rough_array) # vegetation
my_rough_array = np.where(my_rough_array == 29, 0.26, my_rough_array) # woodland shrub
my_rough_array = np.where(my_rough_array == 30, 0.05, my_rough_array) # beaches/ sand
my_rough_array = np.where(my_rough_array == 31, 0.05, my_rough_array) # bare rock
my_rough_array = np.where(my_rough_array == 32, 0.04, my_rough_array) # sparse vegetation
my_rough_array = np.where(my_rough_array == 33, 0.04, my_rough_array) # burnt areas
my_rough_array = np.where(my_rough_array == 34, 0.02, my_rough_array) # glacier/ice
my_rough_array = np.where(my_rough_array == 35, 0.04, my_rough_array) # inland marsh
my_rough_array = np.where(my_rough_array == 36, 0.04, my_rough_array) # peat bog
my_rough_array = np.where(my_rough_array == 37, 0.04, my_rough_array) # salt marsh
my_rough_array = np.where(my_rough_array == 38, 0.01, my_rough_array) # salines
my_rough_array = np.where(my_rough_array == 39, 0.01, my_rough_array) # intertidal flats
my_rough_array = np.where(my_rough_array == 40, 0.01, my_rough_array) # water courses
my_rough_array = np.where(my_rough_array == 41, 0.01, my_rough_array) # water bodies
my_rough_array = np.where(my_rough_array == 42, 0.01, my_rough_array) # coastal lagoon
my_rough_array = np.where(my_rough_array == 43, 0.01, my_rough_array) # estuaries
my_rough_array = np.where(my_rough_array == 44, 0.01, my_rough_array) # sea and ocean

print(my_rough_array.min())
print(my_rough_array.max())

0.0
1.3


In [8]:
#----------------
# replace the bowen values in the array with the correct  values according to the classification in CORINE
#-----------------
my_bowen_array = np.where(my_bowen_array == 1, 1.5, my_bowen_array) # substitute value for continuous urban
my_bowen_array = np.where(my_bowen_array == 2, 0.9, my_bowen_array) # discontinuous urban
my_bowen_array = np.where(my_bowen_array == 3, 1.5, my_bowen_array) # industrial / commercial
my_bowen_array = np.where(my_bowen_array == 4, 1.5, my_bowen_array) # road/rail
my_bowen_array = np.where(my_bowen_array == 5, 1.5, my_bowen_array) # ports
my_bowen_array = np.where(my_bowen_array == 6, 1.5, my_bowen_array) # airports
my_bowen_array = np.where(my_bowen_array == 7, 4.75, my_bowen_array) # mines
my_bowen_array = np.where(my_bowen_array == 8, 1.5, my_bowen_array) # dumps
my_bowen_array = np.where(my_bowen_array == 9, 1.5, my_bowen_array) # construction
my_bowen_array = np.where(my_bowen_array == 10, 0.55, my_bowen_array) # green urban
my_bowen_array = np.where(my_bowen_array == 11, 1.5, my_bowen_array) # sport/leisure
my_bowen_array = np.where(my_bowen_array == 12, 0.55, my_bowen_array) # non-irrigated arable
my_bowen_array = np.where(my_bowen_array == 13, 0.55, my_bowen_array) # irrigated arable
my_bowen_array = np.where(my_bowen_array == 14, 0.55, my_bowen_array) # rice fields
my_bowen_array = np.where(my_bowen_array == 15, 0.55, my_bowen_array) # vineyards
my_bowen_array = np.where(my_bowen_array == 16, 0.55, my_bowen_array) # fruit trees
my_bowen_array = np.where(my_bowen_array == 17, 0.55, my_bowen_array) # olive groves
my_bowen_array = np.where(my_bowen_array == 18, 0.55, my_bowen_array) # pastures
my_bowen_array = np.where(my_bowen_array == 19, 0.55, my_bowen_array) # crops
my_bowen_array = np.where(my_bowen_array == 20, 0.55, my_bowen_array) # complex cultivation
my_bowen_array = np.where(my_bowen_array == 21, 0.55, my_bowen_array) # agriculture
my_bowen_array = np.where(my_bowen_array == 22, 0.7, my_bowen_array) # agro-forest
my_bowen_array = np.where(my_bowen_array == 23, 0.75, my_bowen_array) # broad leaved forest
my_bowen_array = np.where(my_bowen_array == 24, 0.65, my_bowen_array) # evergreen forest
my_bowen_array = np.where(my_bowen_array == 25, 0.7, my_bowen_array) # mix forest
my_bowen_array = np.where(my_bowen_array == 26, 0.8, my_bowen_array) # grassland
my_bowen_array = np.where(my_bowen_array == 27, 0.8, my_bowen_array) # moors
my_bowen_array = np.where(my_bowen_array == 28, 0.8, my_bowen_array) # vegetation
my_bowen_array = np.where(my_bowen_array == 29, 1.25, my_bowen_array) # woodland shrub
my_bowen_array = np.where(my_bowen_array == 30, 1.5, my_bowen_array) # beaches/ sand
my_bowen_array = np.where(my_bowen_array == 31, 1.5, my_bowen_array) # bare rock
my_bowen_array = np.where(my_bowen_array == 32, 0.55, my_bowen_array) # sparse vegetation
my_bowen_array = np.where(my_bowen_array == 33, 0.55, my_bowen_array) # burnt areas
my_bowen_array = np.where(my_bowen_array == 34, 0.5, my_bowen_array) # glacier/ice
my_bowen_array = np.where(my_bowen_array == 35, 0.1, my_bowen_array) # inland marsh
my_bowen_array = np.where(my_bowen_array == 36, 0.1, my_bowen_array) # peat bog
my_bowen_array = np.where(my_bowen_array == 37, 0.1, my_bowen_array) # salt marsh
my_bowen_array = np.where(my_bowen_array == 38, 0.1, my_bowen_array) # salines
my_bowen_array = np.where(my_bowen_array == 39, 0.1, my_bowen_array) # intertidal flats
my_bowen_array = np.where(my_bowen_array == 40, 0.1, my_bowen_array) # water courses
my_bowen_array = np.where(my_bowen_array == 41, 0.1, my_bowen_array) # water bodies
my_bowen_array = np.where(my_bowen_array == 42, 0.1, my_bowen_array) # coastal lagoon
my_bowen_array = np.where(my_bowen_array == 43, 0.1, my_bowen_array) # estuaries
my_bowen_array = np.where(my_bowen_array == 44, 0.1, my_bowen_array) # sea and ocean

print(my_bowen_array.min())
print(my_bowen_array.max())

0.1
255.0


In [9]:
# First of all, gather some information from the original file
[cols,rows] = my_albedo_array.shape
trans = ds.GetGeoTransform()
proj = ds.GetProjection()
albedo_outfile= "/Users/scottlynn73/Documents/CORINE/albedo_CORINE.tif"
bowen_outfile = "/Users/scottlynn73/Documents/CORINEbowen_CORINE.tif"
rough_outfile = "/Users/scottlynn73/Documents/CORINEroughness_CORINE.tif"

In [10]:
#---------------------------
# Create the files, using the geo information from the original file
#--------------------------
outdriver = gdal.GetDriverByName("GTiff")

#--------------------------
# Spit out the albedo geotiff
#--------------------------
outdata   = outdriver.Create(str(albedo_outfile), rows, cols, 1, gdal.GDT_Float32)
outdata.GetRasterBand(1).WriteArray(my_albedo_array)
# Georeference the image
outdata.SetGeoTransform(trans)
# Write projection information
outdata.SetProjection(proj)

#--------------------------
# Spit out the roughness geotiff
#--------------------------
outdata   = outdriver.Create(str(rough_outfile), rows, cols, 1, gdal.GDT_Float32)
# Write the array to the file, which is the original array in this example
outdata.GetRasterBand(1).WriteArray(my_rough_array)
# Georeference the image
outdata.SetGeoTransform(trans)
# Write projection information
outdata.SetProjection(proj)

0

In [11]:
#---------------------------
# Spit out the bowen geotiff
#---------------------------
outdata   = outdriver.Create(str(bowen_outfile), rows, cols, 1, gdal.GDT_Float32)
# Write the array to the file, which is the original array in this example
outdata.GetRasterBand(1).WriteArray(my_bowen_array)
# Georeference the image
outdata.SetGeoTransform(trans)
# Write projection information
outdata.SetProjection(proj)

0